# Utilisation d'un RAG et suivi dans MLflow

# Activation de MLflow

In [ ]:
import mlflow
mlflow.set_tracking_uri("http://mlflow-serveur:8080/")
mlflow.set_experiment("embedding")
mlflow.autolog()

## Les documents

In [1]:
documents = [
    "Baguette is a dish.",
    "Paris is the capital of France.",
    "numpy is a lib for linear algebra",
    "You escaped what I've escaped - You'd be in Paris getting fucked up too",
]

## Embedding des documents et stockage du résultat en mémoire (dans une variable)

In [ ]:
from langchain_community.embeddings import InfinityEmbeddings, InfinityEmbeddingsLocal
#
infinity_api_url = "http://embedding:7997/"
model = "intfloat/multilingual-e5-base"
# model is currently not validated.
embeddings = InfinityEmbeddings(
    model=model, infinity_api_url=infinity_api_url
)
try:
    documents_embedded = embeddings.embed_documents(documents)
    print("embeddings created successful")
except Exception as ex:
    print(
        "Make sure the infinity instance is running. Verify by clicking on "
        f"{infinity_api_url.replace('v1','docs')} Exception: {ex}. "
    )


## Calcul de la similarité entre une question et les documents

In [ ]:
# (demo) compute similarity
import numpy as np
import base64

with mlflow.start_run():
    query = "Where is Paris?"
    query_result = embeddings.embed_query(query)

    scores = np.array(documents_embedded) @ np.array(query_result).T
    print(dict(zip(documents, scores)))

    mlflow.log_param("query", query)
    mlflow.log_param("model", model)
    for doc in dict(zip(documents, scores)):
        print(doc)
        print(dict(zip(documents, scores))[doc])
        mlflow.log_metric("document " + "".join(x for x in doc if x.isalnum() or x ==" "), dict(zip(documents, scores))[doc])